# Decision Tree Implementation
This notebook describes and displays all the code, results and observations of the Decision Tree Implementation from scratch.

*Task 1*

* First it describes the utils.py which has basic functions required for implementing the Decision Tree
* Then it describes base.py which has the basic structure and other important function of the Decision Tree
* Next we have metrics.py which has all the functions to measure and check the predicted results.
* Following it to showcase the usage of Decision Tree on various test cases we have usage.py

*Task 2*
* For task 2 the classification-exp.py file is explained.

*Task 3*
* For task 3 the auto-efficiency.py is explained.

*Task 4*
* For task 4 the experiments.py file is explained.

# Task 1

*utils.py*

In [ ]:

import pandas as pd
import numpy as np

# Function for encoding all types of data(like Categorical data) to numerical data.
def one_hot_encoding(X: pd.DataFrame) -> pd.DataFrame:
    return pd.get_dummies(X)

# Function for checking whether a given series is real or discrete.
# For this a threshold ratio of (unique values/total values) was defined apart from checking the data type of the series.
def check_ifreal(y: pd.Series) -> bool:
    if pd.api.types.is_numeric_dtype(y):
        unique=len(y.unique())
        total=y.count()
        if (unique/total)<0.1:  
            return False
        else:
            return True
    else:
        return False

# For creating the function the formula for entropy was used
def entropy(Y: pd.Series) -> float:
    n=Y.size
    classes=Y.unique()
    H=0
    for i in range(len(classes)):
        # variable for storing the no. of occurences of a class
        m=0
        for j in range(len(Y)):
            if Y.iloc[j]==classes[i]:
                m+=1
        p=m/n
        H+=p*np.log2(p)
    return -1*H

# Mean squared Error- directly uses the formula
def mse(Y: pd.Series) -> float:
    avg=Y.mean()
    sq_err=(Y-avg)**2
    mean_sq_err=sq_err.mean()
    return mean_sq_err

# Gini index- uses the formula of gini index
def gini(y: pd.Series) -> float:
        classes = y.value_counts(normalize=True)
        return 1 - np.sum(classes ** 2)

# Information gain- Takes the output labels Y and input feature attr and the criterion to calculate info gain
# Checks the criterion and accordingly calculates the info gain
def information_gain(Y: pd.Series, attr: pd.Series, criterion: str) -> float:
    Gain=0
    if (criterion=="entropy"):
        HS=entropy(Y)
        Gain+=HS
        c = attr.unique()
        for i in range(len(c)):
            # Si is a list for storing the Y values corresponding to a given class(values of attr.unique)
            Si=[]
            for j in range(len(attr)):
                if attr.iloc[j] == c[i]:
                    Si.append(Y.iloc[j])
            Gain-=(len(Si)/len(Y))*entropy(pd.Series(Si))

        return Gain
    
    elif criterion == "gini_index":
        HS = gini(Y)
        Gain = HS
        c = attr.unique()
        for i in range(len(c)):
            # Si is a list for storing the Y values corresponding to a given class(values of attr.unique)
            Si=[]
            for j in range(len(attr)):
                if attr.iloc[j] == c[i]:
                    Si.append(Y.iloc[j])
            Gain-=(len(Si)/len(Y))*gini(pd.Series(Si))

        return Gain

    elif criterion == "mse":
        HS = mse(Y)
        Gain = HS
        c = attr.unique()
        for i in range(len(c)):
            Si = Y[attr == c[i]]
            Gain -= (len(Si) / len(Y)) * mse(Si)
            
        return Gain



def get_best_split(X: pd.Series, y: pd.Series,  real, criterion: str='information_gain') -> float:
    best_split_value = None
    best_score = -float('inf') 

    if (real):
        sorted_indices = np.argsort(X)
        sorted_X = X.iloc[sorted_indices]
        sorted_y = y.iloc[sorted_indices]
        
        for i in range(1, len(sorted_X)):
            split_value = (sorted_X.iloc[i-1] + sorted_X.iloc[i]) / 2
            
            left = sorted_X <= split_value
            if (criterion=='information_gain'):
                info_gain = information_gain(y, left, 'mse')
            else:
                info_gain = information_gain(y, left, 'mse')

            if info_gain > best_score:
                best_score = info_gain
                best_split_value = split_value
    else:
        if (criterion=='information_gain'):
            best_score = information_gain(y, X, 'mse')
        else:
            best_score = information_gain(y, X, 'mse')
        
    return best_split_value, best_score


def get_best_val(X: pd.Series, y: pd.Series, real, criterion: str='information_gain') -> str:
    best_val = None
    best_info_gain = -float('inf')
    
    if (real==0):
        if (criterion=='information_gain'):
            best_info_gain=information_gain(y, X, 'entropy')
        else:
            best_info_gain=information_gain(y, X, 'gini_index')
        
    else:
        sorted_indices = np.argsort(X)
        sorted_X = X.iloc[sorted_indices]
        sorted_y = y.iloc[sorted_indices]
        
        for i in range(1, len(sorted_X)):
            split_value = (sorted_X.iloc[i-1] + sorted_X.iloc[i]) / 2
            
            left = sorted_X <= split_value
            if (criterion=='information_gain'):
                info_gain = information_gain(y, left, 'entropy')
            else:
                info_gain = information_gain(y, left, 'gini_index')

            if info_gain > best_info_gain:
                best_info_gain = info_gain
                best_val = split_value
            
    return best_val, best_info_gain


def opt_split_attribute(X: pd.DataFrame, y: pd.Series, features: pd.Series, real_target: bool, real_feature, criterion: str):
    """
    Function to find the optimal attribute to split about.
    If needed you can split this function into 2, one for discrete and one for real valued features.
    You can also change the parameters of this function according to your implementation.

    features: pd.Series is a list of all the attributes we have to split upon

    return: attribute to split upon
    """

    # According to whether the features are real or discrete valued and the criterion, find the attribute from the features series with the maximum information gain (entropy or variance based on the type of output) or minimum gini index (discrete output).
    best_feature=None
    best_split=None
    best_info_gain=-float('inf')
    if real_target:
        c = 0
        for i in features:
            split, info_gain = get_best_split(X.loc[:,i], y, real_feature[c], criterion)
            c += 1
            if info_gain>best_info_gain:
                best_info_gain=info_gain
                best_split=split
                best_feature=i
    else:
        c = 0
        for i in features:
            split, info_gain = get_best_val(X.loc[:,i], y, real_feature[c], criterion)
            c += 1
            if info_gain>best_info_gain:
                best_info_gain=info_gain
                best_split=split
                best_feature=i
    return best_feature, best_split


def split_data(X: pd.DataFrame, y: pd.Series, attribute, value):
    """
    Function to split the data according to an attribute.
    Handles both discrete and real-valued features.
    """

    if value is None:
        # For discrete features
        splits = []
        unique_values = X[attribute].unique()
        
        for u in unique_values:
            X_split = X[X[attribute] == u]
            y_split = y[X[attribute] == u]
            splits.append((X_split, y_split))
    else:
        # For real-valued features
        X_split_left = X[X[attribute] <= value]
        y_split_left = y[X[attribute] <= value]
        
        X_split_right = X[X[attribute] > value]
        y_split_right = y[X[attribute] > value]
        
        splits = [(X_split_left, y_split_left), (X_split_right, y_split_right)]
        
    return splits
